<a href="https://colab.research.google.com/github/kathuman/Python-Experiments/blob/main/201228_Indicators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://www.youtube.com/watch?v=fJ3CfEwr39k
# Description: Create and plot multiple technical indicators

In [3]:
# Import the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas_datareader as pdr

In [6]:
start_date = '2019-11-25'
end_date = '2020-11-23'
df = pdr.DataReader('TSLA', 'yahoo', start_date, end_date)

In [7]:
df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2019-11-25,68.914001,66.891998,68.863998,67.267998,61697500.0,67.267998
2019-11-26,67.099998,65.419998,67.054001,65.783997,39737000.0,65.783997
2019-11-27,66.786003,65.713997,66.223999,66.258003,27778000.0,66.258003
2019-11-29,66.251999,65.500000,66.222000,65.987999,12328000.0,65.987999
2019-12-02,67.276001,65.737999,65.879997,66.973999,30372500.0,66.973999


In [13]:
# check the dataframe index
df.index

DatetimeIndex(['2019-11-25', '2019-11-26', '2019-11-27', '2019-11-29',
               '2019-12-02', '2019-12-03', '2019-12-04', '2019-12-05',
               '2019-12-06', '2019-12-09',
               ...
               '2020-11-10', '2020-11-11', '2020-11-12', '2020-11-13',
               '2020-11-16', '2020-11-17', '2020-11-18', '2020-11-19',
               '2020-11-20', '2020-11-23'],
              dtype='datetime64[ns]', name='Date', length=252, freq=None)

In [ ]:
# Create functions to calculate SMA and EMA

# Create the SImple Moving Average (SMA)
def SMA(data, period=20, column='Close'):
  return dat[column].rolling(window=period).mean()